### **本节介绍神经网络块** 

块是由层组成的，nn.sequential(Moudle,Moudle,....)就是一个块


**一，下面来实现自定义块**

In [23]:
# 导入包
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F

In [24]:
# nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
# 自定义块
class MLP(nn.Module):
    """实现等价于nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))的块"""
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)

    def forward(self,x):
        """我们只需要定义前向传播即可"""
        return self.out(F.relu(self.hidden(x)))

In [25]:
net = MLP()
x = torch.rand(2,20)
net(x)

tensor([[ 0.0660,  0.1153,  0.2344,  0.0596, -0.0332,  0.1688, -0.0112, -0.0604,
          0.1044,  0.0620],
        [ 0.0563,  0.0160,  0.2229, -0.0156, -0.1508,  0.1522,  0.0249, -0.1101,
          0.1094, -0.0395]], grad_fn=<AddmmBackward0>)

**二，顺序块**

In [26]:
# 下面实现自己的顺序块
class MySequential(nn.Module):
    """手搓的nn.sequential"""
    def __init__(self,*args):
        super().__init__()
        for idx,moudle in enumerate(args):
            self._modules[str(idx)] = moudle # _modules内部的实现是一个有序字典

    def forward(self,x):
        # 有序字典确保了层的顺序访问(注意要访问字典的values,而非keys)
        for block in self._modules.values():
            x = block(x)
        return x

In [27]:
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
x = torch.rand(2,20)
net(x)

tensor([[ 0.0591, -0.0725, -0.0099, -0.0648,  0.1009,  0.1441, -0.1680, -0.1707,
          0.0236,  0.1026],
        [ 0.1303, -0.0668, -0.0340, -0.0432,  0.0595,  0.0940, -0.1412, -0.1317,
         -0.0146,  0.2491]], grad_fn=<AddmmBackward0>)

**三，在前向传播中添加执行代码**

In [39]:
class Testnet(nn.Module):
    """实际中可能不会用，仅用作测试"""
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand(20,20,requires_grad = False)
        self.linear = nn.Linear(20,20)

    def forward(self,x):
        print("Hello,World!") # 嵌入自己的代码
        x = x.mm(self.rand_weight) + 1
        x = F.relu(x)
        x = self.linear(x)
        while x.abs().sum() > 1:
            x /= 2
        return x.sum()

In [40]:
net = Testnet()
x = torch.rand(2,20)
net(x)

Hello,World!


tensor(-0.2671, grad_fn=<SumBackward0>)

**四，混合块**

In [43]:
class NetCommbat(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(20,20),nn.ReLU(),
            nn.Linear(20,20),nn.ReLU(),
        )
        self.layer2 = nn.Linear(20,20)

    def forward(self,x):
        x = self.layer1(x)
        x = F.relu(self.layer2(x))
        return x

In [44]:
net = nn.Sequential(
    NetCommbat(),
    nn.Linear(20,20),
    Testnet()
)
x = torch.rand(2,20)
net(x)

Hello,World!


tensor(0.1123, grad_fn=<SumBackward0>)